In [1]:
#API Keys
CLIENT_ID = 'lpakYt45RJlvK6Lp6ybehw'
SECRET_KEY = 'XXHGvxgxAhlEHK6-KMy8nP_QmcedXg'

In [2]:
#imports the request module and acquires an access token from the api by giving it our user name and password for reddit which we can then use to access and use the API
import requests

auth = requests.auth.HTTPBasicAuth(CLIENT_ID, SECRET_KEY)
data = {
    'grant_type' : 'password',
    'username' : 'PiccoloNo3695',
    'password' : 'scooby2203'
}

headers = {'User-Agent' : 'MyAPI/0.0.1'}
res = requests.post('https://www.reddit.com/api/v1/access_token', auth=auth, data=data, headers=headers)
TOKEN = res.json()['access_token']
headers['Authorization'] = f'bearer {TOKEN}'

In [3]:
#checks to make sure api is connecting properly, response 200 means all is well
requests.get('https://oauth.reddit.com/api/v1/me', headers=headers)

<Response [200]>

In [4]:
#gets all the hot posts from the python subreddit use the paramas limit to change the amount of posts you are getting (100 is the max), also change the url at the end to either new/hot/top/rising depending on what you want from the subreddit
res = requests.get('http://oauth.reddit.com/r/python/top/?t=all', headers=headers, params={'limit' : '100'})
res.json()


{'kind': 'Listing',
 'data': {'after': 't3_hei6kg',
  'dist': 100,
  'modhash': None,
  'geo_filter': '',
  'children': [{'kind': 't3',
    'data': {'approved_at_utc': None,
     'subreddit': 'Python',
     'selftext': '',
     'author_fullname': 't2_g532u',
     'saved': False,
     'mod_reason_title': None,
     'gilded': 0,
     'clicked': False,
     'title': "Lad wrote a Python script to download Alexa voice recordings, he didn't expect this email.",
     'link_flair_richtext': [{'e': 'text', 't': 'Discussion'}],
     'subreddit_name_prefixed': 'r/Python',
     'hidden': False,
     'pwls': 6,
     'link_flair_css_class': 'discussion',
     'downs': 0,
     'thumbnail_height': 68,
     'top_awarded_type': None,
     'hide_score': False,
     'name': 't3_g53lxf',
     'quarantine': False,
     'link_flair_text_color': 'dark',
     'upvote_ratio': 0.98,
     'author_flair_background_color': None,
     'ups': 12339,
     'total_awards_received': 27,
     'media_embed': {},
     'thum

In [5]:
#this sorts through the request json, picking out the important information and converting it into a dataframe table format where it can be better interpreted
import pandas as pd

df = pd.DataFrame()
df = df.loc[:, ~df.columns.str.contains('Unnamed')]
for post in res.json()['data']['children']:
    name = post ['data']['name']
    link = name[3:]
    df = df.append({
        'Question' : post['data']['selftext'],
        'QuestionAlternative' : post['data']['title'],
        'NFSW' : post['data']['over_18'],
        'Name' : link,
        'Source' : 'Reddit ' + "- r/" + post ['data']['subreddit'],
        'Number of Comments' : post['data']['num_comments']
    }, ignore_index=True)
df


,Question,QuestionAlternative,NFSW,Name,Source,Number of Comments
0,,Lad wrote a Python script to download Alexa vo...,0.0,g53lxf,Reddit - r/Python,149.0
1,"9777 upvotes,\n\n967 downvotes\n\nand 452 comm...",This post has:,0.0,hoolsm,Reddit - r/Python,442.0
2,,I redesign the Python logo to make it more modern,0.0,gftejm,Reddit - r/Python,270.0
3,,Automate the boring stuff with python - tinder,0.0,7kpme8,Reddit - r/Python,337.0
4,,Just finished programming and building my own ...,0.0,dmkx8a,Reddit - r/Python,472.0
...,...,...,...,...,...,...
95,,Work laptop doesn't allow minesweeper? I'm gon...,0.0,es8dai,Reddit - r/Python,116.0
96,,"The project is finally done, thanks a lot for ...",0.0,fbyvl5,Reddit - r/Python,149.0
97,,I created a video about Neural Networks that i...,0.0,l8jvev,Reddit - r/Python,62.0
98,,I created a python program for interactive bat...,0.0,dqminb,Reddit - r/Python,121.0
